In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from scipy.sparse import csr_matrix
import implicit

/Users/platon.fedorov/opt/anaconda3/envs/hh_recsys/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [27]:
train_path = "../data/processed/train.parquet"
test_path  = "../data/processed/test_inference.parquet"
user_history_data_path = "dumps/user_history/history_data.parquet"

In [19]:
train = pd.read_parquet(train_path)

In [4]:
train.head()

,user_id,session_id,vacancy_id,action_type,action_dt,is_session_has_more_then_one_date,session_day
0,u_332060,s_28301374,"[v_2571684, v_488179, v_2389179, v_1393783, v_...","[2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 2, 1, 2, 2, 2, ...","[2023-11-01T00:40:58.105000000, 2023-11-01T00:...",False,1
1,u_1057881,s_33868982,[v_665861],[2],[2023-11-01T00:23:51.452000000],False,1
2,u_1036784,s_32474802,[v_2594840],[2],[2023-11-01T00:52:34.023000000],False,1
3,u_786220,s_14060785,"[v_1473781, v_1622905, v_1621959, v_2289180, v...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 2, ...","[2023-11-01T00:58:20.793000000, 2023-11-01T01:...",False,1
4,u_639152,s_23205986,"[v_695738, v_22433, v_1590524, v_502496, v_200...","[2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, ...","[2023-11-01T01:14:20.828000000, 2023-11-01T00:...",False,1


In [5]:
pairs = train[['user_id', 'vacancy_id', 'action_type']].explode(['vacancy_id', 'action_type']).reset_index(drop=True)
pairs

,user_id,vacancy_id,action_type
0,u_332060,v_2571684,2
1,u_332060,v_488179,2
2,u_332060,v_2389179,2
3,u_332060,v_1393783,2
4,u_332060,v_2608935,2
...,...,...,...
13524544,u_967286,v_1185617,2
13524545,u_53297,v_237611,2
13524546,u_1174879,v_1394023,2
13524547,u_938734,v_1102303,2


In [6]:
user_counts = pairs.groupby("user_id").count().reset_index()

In [7]:
used_users = user_counts[user_counts["vacancy_id"] >= 10]["user_id"]

In [8]:
used_pairs = pairs.merge(used_users, "inner", "user_id")

In [9]:
item_counts = used_pairs.groupby("vacancy_id").count().reset_index()

In [10]:
used_items = item_counts[item_counts["user_id"] >= 15]["vacancy_id"]

In [11]:
used_pairs = pairs.merge(used_items, "inner", "vacancy_id")

In [12]:
def sparcity(pairs):
    return len(pairs) / (len(pairs["user_id"].unique()) * len(pairs["vacancy_id"].unique()))

In [13]:
sparcity(pairs), sparcity(used_pairs)

(1.670173463413874e-05, 9.026066338405871e-05)

In [14]:
unique_users = used_pairs["user_id"].unique().tolist()
unique_vacancies = used_pairs['vacancy_id'].explode().unique().tolist()

user2idx = {user_id: idx for idx, user_id in enumerate(unique_users)}
vac2idx = {vac_id: idx for idx, vac_id in enumerate(unique_vacancies)}
idx2vac = {idx: vac_id for vac_id, idx in vac2idx.items()}

action_weights = {
    1: 4.0,
    2: 1.0,
    3: 2.0
}

In [15]:
users = used_pairs['user_id'].map(user2idx).to_numpy()
vacancies = used_pairs['vacancy_id'].map(vac2idx).to_numpy()
preferences = used_pairs['action_type'].map(action_weights).to_numpy()

In [16]:
uv_mat = csr_matrix((preferences, (users, vacancies)))

In [17]:
!OPENBLAS_NUM_THREADS=1

In [18]:
als_model = implicit.als.AlternatingLeastSquares(
    alpha=40,
    factors=50,
    random_state=137,
    iterations=15,
    calculate_training_loss=True,
    regularization=0.001
)

/Users/platon.fedorov/opt/anaconda3/envs/hh_recsys/lib/python3.8/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 8 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


In [20]:
als_model.fit(uv_mat)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [06:56<00:00, 27.79s/it, loss=0.00277]


In [82]:
type(als_model)

implicit.cpu.als.AlternatingLeastSquares

In [21]:
als_model.user_factors.shape, als_model.item_factors.shape

((582823, 50), (178379, 50))

In [28]:
test = pd.read_parquet(test_path)
user_history_data = pd.read_parquet(user_history_data_path)

In [29]:
test = test.merge(user_history_data, "left", "user_id")

In [30]:
test_users = list(filter(lambda x: x in user2idx, test["user_id"].unique().tolist()))
print("Пользователей в тесте, для которых есть ALS", len(test_users))
print("Это", len(test_users) / len(test) * 100, "% от всех юзеров теста")

Пользователей в тесте, для которых есть ALS 35124
Это 75.79955975657128 % от всех юзеров теста


In [43]:
predictions = []

def recommend_by_als(user_id, vacancy_id, items, als):
    if user_id not in user2idx:
        return []

    if items in 

    user_idx = user2idx[user_id]
    recs = als.recommend(
        user_idx,
        uv_mat[cuser],
        N=100,
        recalculate_user=True,
        filter_already_liked_items=True
    )[0]
    recs = [idx2vac[cv] for cv in recs]

    return recs
    

test["predictions"] = test[["user_id", "vacancy_id", "items"]].apply(
    lambda row: recommend_by_als(row['user_id'], row['vacancy_id'], row['items'], als_model),
    axis=1
)

In [64]:
from itertools import chain

def merge_history(vacancy_id, items):
    if isinstance(items, np.ndarray) or isinstance(items, list):
        vacancy_id = list(chain(*items)) + list(vacancy_id)
    return vacancy_id

def prepare_one_step_matrix(test, vac2idx, action_weights):
    test["history"] = test[["vacancy_id", "items"]].apply(
        lambda row: merge_history(row["vacancy_id"], row["items"]),
        axis=1
    )
    test["history_actions"] = test[["action_type", "item_actions"]].apply(
        lambda row: merge_history(row["action_type"], row["item_actions"]),
        axis=1
    )
    pairs = test[['user_id', 'history', 'history_actions']]\
        .explode(['history', 'history_actions'])\
        .reset_index(drop=True)
    used_items = pd.Series(vac2idx.keys())
    used_items.name = "history"
    one_step_pairs = pairs.merge(used_items, "inner", "history")

    print(len(pairs) / (len(pairs["user_id"].unique()) * len(pairs["history"].unique())))

    unique_users = one_step_pairs["user_id"].unique().tolist()
    user2idx = {user_id: idx for idx, user_id in enumerate(unique_users)}

    users = one_step_pairs['user_id'].map(user2idx).to_numpy()
    vacancies = one_step_pairs['history'].map(vac2idx).to_numpy()
    preferences = one_step_pairs['history_actions'].map(action_weights).to_numpy()

    return csr_matrix((preferences, (users, vacancies))), user2idx

In [58]:
test.head(1)

,user_id,session_id,target_session_id,vacancy_id,action_type,action_dt,target_vacancy_id,items,item_actions,predictions,history,history_actions
0,u_1000060,s_19856666,s_6481076,[v_1962314],[2],[2023-11-10T14:21:18.628000000],v_76636,"[[v_1500295, v_1500295], [v_1500295, v_524850]...","[[2, 1], [2, 2], [2], [2, 2], [2], [2, 2, 2], ...","[v_1840884, v_1508872, v_663937, v_2384625, v_...","[v_1500295, v_1500295, v_1500295, v_524850, v_...","[2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."


In [65]:
one_step_matrix, user2idx = prepare_one_step_matrix(test, vac2idx, action_weights)

0.0001058994958860454


In [75]:
I, D = als_model.recommend(
    np.array([i for i in range(len(user2idx))]),
    one_step_matrix,
    N=100,
    recalculate_user=True,
    filter_already_liked_items=False
)

In [76]:
test["predictions"] = test["user_id"]\
    .apply(
        lambda user_id: list(map(lambda x: idx2vac[x], I[user2idx[user_id]]))
            if user_id in user2idx else []
    )

In [79]:
als_model_2 = implicit.als.AlternatingLeastSquares(
    alpha=40,
    factors=50,
    random_state=137,
    iterations=15,
    calculate_training_loss=True,
    regularization=0.001
)

als_model_2.item_factors = als_model.item_factors

I2, D2 = als_model_2.recommend(
    np.array([i for i in range(len(user2idx))]),
    one_step_matrix,
    N=100,
    recalculate_user=True,
    filter_already_liked_items=False
)

In [80]:
test["predictions_2"] = test["user_id"]\
    .apply(
        lambda user_id: list(map(lambda x: idx2vac[x], I2[user2idx[user_id]]))
            if user_id in user2idx else []
    )

In [81]:
def reciprocal_rank(row):
    predictions = row["predictions_2"]
    target = row["target_vacancy_id"]

    for idx, prediction in enumerate(predictions):
        if prediction == target:
            return 1 / (idx + 1)

    return 0

def recall(row):
    predictions = row["predictions_2"]
    target = row["target_vacancy_id"]

    return int(target in set(predictions))

mrr = test[["predictions_2", "target_vacancy_id"]].apply(reciprocal_rank, axis=1).mean()
recall = test[["predictions_2", "target_vacancy_id"]].apply(recall, axis=1).mean()

print("mrr:", mrr)
print("mean recall:", recall)

mrr: 0.015228182967661659
mean recall: 0.12167119858431524


In [ ]:
"""

Обучили ALS + рекомендуем, кому можем без обновления эмбеддов юзеров (Для остальных [])
mrr: 0.009213898005761204
mean recall: 0.08664594932884458

mrr: 0.015228182967661659
mean recall: 0.12167119858431524

Обучили ALS + рекомендуем, кому можем без обновления эмбеддов юзеров (Для остальных []), фильтруя саму матрицу
mrr: 0.007645883133958031
mean recall: 0.07119426820320256

Обучили ALS + Сделали One-step на данных инференса (Если не было эмбедов айтемов из обучения, то []), фильтруя саму матрицу
mrr: 0.00988495641195503
mean recall: 0.08455263498640425

"""

In [123]:
"""
    Соберём i2i списочки

    И сохраним эмбеддинги
"""

"""
    Эмбеддинги
"""
import pickle

# embeddings_path = "dumps/als/item_factors/embeddings.pickle"
# embedding_names_path = "dumps/als/item_factors/embeddings_names.parquet"
# pd.DataFrame({"vacancy_id": [idx2vac[i] for i in range(len(als_model.item_factors))]}).to_parquet(embedding_names_path)
# pickle.dump(als_model.item_factors, open(embeddings_path, 'wb+'), protocol=4)

# i2i_path =  "dumps/als/als_i2i.parquet"
# i2i_ids, i2i_scores = als_model.similar_items([i for i in range(len(als_model.item_factors))], 101)
# vacancy_ids_ = []
# neighbours_  = []
# for vacancy_als_idx, (i, d) in enumerate(zip(i2i_ids, i2i_scores)):
#     vacancy_ids_.append(idx2vac[vacancy_als_idx])
#     neighbours_.append(list(map(lambda x: idx2vac[x[0]], filter(lambda x: x[1] >= 0.9 and x[0] != vacancy_als_idx, zip(i, d)))))
# als_i2i = pd.DataFrame.from_dict({
#     "vacancy_id": vacancy_ids_,
#     "neighbours": neighbours_
# })
# als_i2i = als_i2i[als_i2i["neighbours"].apply(len) > 0]
# als_i2i.to_parquet(i2i_path)
# als_i2i.shape[0]

174890

In [126]:
# als_i2i["neighbours"].apply(len).describe()


In [82]:
vacancies = pd.read_parquet("dumps/production/i2i/name_area/vacancies.parquet")

In [86]:
vacancy_2_name = {row["vacancy_id"]: row["text"] for _, row in vacancies.iterrows()}

In [103]:
for row_idx, row in vacancies.iterrows():
    vacancy_id = row["vacancy_id"]
    vacancy_name = vacancy_2_name[vacancy_id]

    if vacancy_id not in vac2idx:
        continue

    if vacancy_name.startswith("Москва"):
        continue

    als_vacacny_index = vac2idx[vacancy_id]
    ids, scores= als_model.similar_items(als_vacacny_index, 5)

    print("-=-=--=-===-=")
    print(vacancy_name)
    print("^^^^^^^^^^^^^")

    neighbours = list(map(lambda z: vacancy_2_name[z], filter(lambda y: y in vacancy_2_name, map(lambda x: idx2vac[x], ids))))
    # print(pd.DataFrame({"neighbours": neighbours, "score": scores}))

    for score, neighbour in zip(scores, neighbours):
        print("    ", score, neighbour)

    if row_idx >= 2000:
        break

-=-=--=-===-=
Приморская область. Менеджер по работе с локальными ключевыми клиентами (ДФО)
^^^^^^^^^^^^^
     1.0000001 Приморская область. Менеджер по работе с локальными ключевыми клиентами (ДФО)
     0.9951988 Приморская область. Менеджер по работе с ключевыми клиентами в регионе Дальний Восток
     0.994969 Приморская область. Менеджер по ключевым клиентам (ГРИН АГРО)
     0.994867 Приморская область. Руководитель отдела продаж
     0.9945531 Приморская область. Менеджер по работе с партнерами/Региональный менеджер (Дальневосточный Федеральный округ)
-=-=--=-===-=
Екатеринбург. Кладовщик на детские новогодние подарки
^^^^^^^^^^^^^
     1.0000001 Екатеринбург. Кладовщик на детские новогодние подарки
     0.9984795 Екатеринбург. Рабочий на производство
     0.99842525 Екатеринбург. Продавец (без кассы)
     0.9983604 Екатеринбург. Слесарь-сборщик РЭА и приборов
     0.9982873 Екатеринбург. Рабочий на производство
-=-=--=-===-=
Московская область. Сборщик с ежедневной выплатой Wildbe